In [23]:
import sys
import json
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [24]:
def get_filepath(path,filename):
    '''
    get filePath cas insensitive (ex: read demand.csv but file is name DEMAND.csv)
    path:'path/ ex: '../../scenarios/base/inputs/'
    filename: ex: demand.csv
    '''
    files = os.listdir(path)
    file = [file for file in files if filename.lower() == file.lower()]
    if len(file)==0:
        print(f'{path+filename} does not exist')
        return path+filename
    return path+file[0]

In [61]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [26]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')


In [67]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())

# inputs

In [42]:
links = gpd.read_file(pt_folder + 'links.geojson', engine=io_engine) 
nodes = gpd.read_file(pt_folder + 'nodes.geojson', engine=io_engine)
links = links.set_index('index')
nodes = nodes.set_index('index')

In [43]:
rnodes = gpd.read_file(road_folder + 'road_nodes.geojson', engine=io_engine)
rnodes = rnodes.set_index('index')
rlinks = gpd.read_file(road_folder + 'road_links.geojson', engine=io_engine)
rlinks = rlinks.set_index('index')

In [44]:
filepath = get_filepath(input_folder,'zones.geojson')
zones = gpd.read_file(filepath)
zones = zones.set_index('index')

# split quenedi road links


In [45]:
from quetzal.engine.road_model import RoadModel, _reverse_geom

self = RoadModel(rlinks,rnodes,zones,ff_time_col='time')
self.split_quenedi_rlinks()
rlinks = self.road_links

del self

# create Model

In [46]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.road_links = rlinks
sm.road_nodes = rnodes
sm.zones = zones

In [47]:
centroid = [*LineString(sm.nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

32615

In [48]:
sm = sm.change_epsg(crs,'meter')

Reprojecting model from epsg 4326 to epsg 32615: 100%|█| 11/11 [00:05<00:00,  2.


# clustering and walkmodel

In [49]:
sm.preparation_clusterize_nodes(distance_threshold=200)


In [50]:
#sm._add_type_prefixes({'nodes': 'node_','links':'link_','road_nodes':'road_node','road_links':'road_link_','zones':'zone_'})
sm._add_type_prefixes({'zones':'zone_'})

In [51]:
sm.road_links['walk_time'] = sm.road_links['length'] / var['footpaths']['wor_speed'] * 3.6


In [52]:
sm.preparation_footpaths(max_length=var['footpaths']['max_length'],
                         speed=var['footpaths']['speed'])


In [53]:
# Zone to transit
sm.preparation_ntlegs(
    short_leg_speed = var['zone_to_transit']['speed'],
    long_leg_speed = var['zone_to_transit']['speed'], # tout le monde marche
    threshold = 1000,
    zone_to_transit = True,
    zone_to_road = True,
    road_to_transit = True,
    n_ntlegs = var['zone_to_transit']['nbr'],
    max_ntleg_length = var['zone_to_transit']['max_length'],
)
sm._add_type_prefixes({'zone_to_transit':'ztt_'})

In [54]:
sm.integrity_test_all(errors='ignore', verbose=True)

failed: integrity_test_circular_lines
passed: integrity_test_collision


47435it [00:00, 70006.92it/s]


passed: integrity_test_dead_ends
passed: integrity_test_duplicate_volumes
passed: integrity_test_isolated_roads
passed: integrity_test_nodeset_consistency
failed: integrity_test_road_duplicated_ab_links


47435it [00:02, 16487.30it/s]


failed: integrity_test_road_network
passed: integrity_test_road_nodeset_consistency
passed: integrity_test_sequences
passed: integrity_test_str_collision


In [81]:
sm.road_links['dup'] = sm.road_links['a'] + sm.road_links['b']
sm.road_links = sm.road_links.drop_duplicates('dup').drop(columns=['dup'])

In [82]:
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

sm.to_zippedpickles(model_folder +'network')  

broken_sequences: 100%|█████████████████████████| 25/25 [00:04<00:00,  6.01it/s]
